# Cortical Magnification Tutorial


**Author**: &nbsp;&nbsp; [Noah C. Benson](mailto:nben@nyu.edu)  
**Date**: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Jan. 10, 2020  
**Link**: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [noahbenson/cortical-magnification-tutorial](https://github.com/noahbenson/cortical-magnification-tutorial)


## Introduction

This notebook is intended as an introduction to cortical magnification and how it is calculated. When run in its relevant docker-image, with correctly configured HCP credentials (or with additional data that has been configured correctly), it should be possible to use this notebook to calculate and export cortical magnification values for any subject whose retinotopic maps (pRF parameters) have been solved via a pRF solver such as [analyzePRF](https://kendrickkay.net/analyzePRF/), [Vistasoft](https://github.com/vistalab/vistasoft), or [Popeye](https://github.com/kdesimone/popeye).

### What is cortical magnification?

The cortical magnification function (CMF) is the function $f(\mathbf{u}) = m$ where $\mathbf{u}$ is the $(x,y)$ position in the visual field (typically in degrees of visual angle), and $m$ is the number of mm$^2$ of cortex per deg$^2$ of the visual field at that point in the visual field. Accordingly, a high cortical magnification, e.g. $m(\mathbf{u})$ = 150 mm$^2$/deg$^2$, indicates that there is a lot of cortex devoted to a relatively small amount of the visual field, while a low value indicates that very little cortical surface is devoted to that part of the visual field. In the human visual system, the fovea tends to have a cortical magnification that is many times larger than that of the periphery.

Occasionally, cortical magnification is also reported in terms of mm/deg--linear cortical magnification. This is the form a measurement would take if, for example, we measured the length (in mm) of the 5° iso-eccentricity contour across the surface of V1 then divided it by the length of the equivalent 5° iso-eccentricity arc in the visual field ($\pi/2 \times 5° \approx 7.85°$). When cortical magnification is in units of mm/deg, it typically is measuring the magnification in a particular direction such as tangential (around the visual field) or radial (toward/from the fovea). In the case of the 5° iso-eccentricity arc, the direction would be tangential.

### Tutorial Setup

#### Import and Configure Libraries

To run this tutorial, we must have a few things configured. First, we should start by importing a few relevant libraries.

In [1]:
# Import some standard/utility libraries:
import os, sys, time, h5py, zipfile
import six           # six provides python 2/3 compatibility

# Import our numerical/scientific libraries, scipy and numpy:
import numpy as np
import scipy as sp

# The pimms (Python Immutables) library is a utility library that enables lazy
# computation and immutble data structures; https://github.com/noahbenson/pimms
import pimms

# The neuropythy library is a swiss-army-knife for handling MRI data, especially
# anatomical/structural data such as that produced by FreeSurfer or the HCP.
# https://github.com/noahbenson/neuropythy
import neuropythy as ny

# Import graphics libraries:
# Matplotlib/Pyplot is our 2D graphing library:
import matplotlib as mpl
import matplotlib.pyplot as plt
# We also use the 3D graphics library ipyvolume for 3D surface rendering
import ipyvolume as ipv

##### Configure matplotlib for 2D plotting:

In [2]:
# These "magic commands" tell matplotlib that we want to plot figures inline and
# That we are using qt as a backend; due to bugs in certain versions of
# matplotlib, we put them in a separate cell from the import statements above
# and the configuration statements below.
%gui qt
%matplotlib inline

In [3]:
# Additional matplotlib preferences:
font_data = {'family':'sans-serif',
             'sans-serif':['Helvetica Neue', 'Helvetica', 'Arial'],
             'size': 10,
             'weight': 'light'}
mpl.rc('font',**font_data)
# we want relatively high-res images, especially when saving to disk.
mpl.rcParams['figure.dpi'] = 72*2
mpl.rcParams['savefig.dpi'] = 72*4

##### Configure neuropythy's HCP interface, if not done automatically:

Note: If you intend to use a custom subject and set of pRF measurements with this notebook instead of using an HCP subject, you can skip this section.

The [neuropythy](https://github.com/noahbenson/neuropythy) library can easily be configured to automatically download HCP data it is requested. In order to do this, however, it must have been given a set of HCP credentials. The HCP uses the Amazon S3 so these credentials are in the form of a "key" and a "secret". To obtain HCP credentials, you must register at the [HCP database website](https://db.humanconnectome.org/) then generate Amazon S3 credentials through their interface. The [neuropythy configuration documentation](https://github.com/noahbenson/neuropythy/wiki/Configuration) explains how to do this in more detail.

Your credentials will look something like "`AKAIG8RT71SWARPYUFUS`" and "`TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4`" (key and secret). They are often printed with a "`:`" between them like "`AKAIG8RT71SWARPYUFUS:TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4`". If, when you started up the docker-container running this notebook, you provided your credentials as a `:`-separated string like this via the environment variable HCP_CREDENTIALS, you should be fine. If neuropythy found your credentials, they should be in neuropythy's `config` structure, which behaves like a Python dictionary. However, if you are running this tutorial in the Neurohackademy 2019 class itself, you likely were not able to do this and will have to set these credentials manually. If neuropythy could not read your credentials or if they were not set, then `ny.config['hcp_credentials']` will be `None`. If this is the case, you can either:
 * set the credentials directly in this notebook by running something like:  
   ```python
   key = 'AKAIG8RT71SWARPYUFUS'
   secret = 'TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4'
   ny.config['hcp_credentials'] = (key, secret)
   ```
 * restart the docker container after configuring the `HCP_CREDENTIALS` environment variable:
   ```bash
   > export HCP_CREDENTIALS="AKAIG8RT71SWARPYUFUS:TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4"
   > docker-compose up
   ```
 * store the credentials in a local file and import its contents into the `HCP_CREDENTIALS` environment variable:
   ```bash
   > echo "AKAIG8RT71SWARPYUFUS:TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4" > ~/.hcp-passwd
   > export HCP_CREDENTIALS="`cat ~/.hcp-passwd`"
   > docker-compose up
   ```

We also want to check that neuropythy was able to connect to the HCP database. If this fails, it is possible that either your credentials are incorrect/expired or that you do not have a valid internet connection, or that you did something unexpected when mounting volumes into the docker that prevent neuropythy from knowing where to store the HCP data (unlikely).

In [5]:
# Check that HCP credentials were found:
if ny.config['hcp_credentials'] is None:
    raise Exception('No valid HCP credentials were found!\n'
                    'See above instructions for remedy.')

# Check that we can access the HCP database:
# To do this we grab the 's3fs' object from neuropythy's 'hcp' dataset; this
# object maintains a connection to Amazon's S3 using the hcp credentials. We use
# it to perform a basic ls operation on the S3 filesystem. If this fails, we do
# not have a working connection to the S3.
try: files = ny.data['hcp'].s3fs.ls('hcp-openaccess')
except Exception: files = None
if files is None:
    raise Exception('Could not communicate with S3!\n'
                    'This probably indicates that your credentials are wrong'
                    ' or that you do not have an internet connection.')

print('Configuration appears fine!')

Exception: No valid HCP credentials were found!
See above instructions for remedy.

Setup a subject and the data we need for cortical magnification. Because we are using an HCP subject, most of this is done for us.

In [ ]:
sid = 177746
sub = ny.hcp_subject(sid)

##### Alternately, configure a custom data source

Note: If you intend to use this notebook with an HCP subject rather than with your own subject, you can skip this section.

As an alternative to using HCP data, you can configure this notebook to work with a separate data source. If you are using this notebook via the docker-image or the docker-compose command, you will have to make sure that the data you wish to load is already mounted into the docker-image when it is run. If you are using `docker-compose up` or `docker run` to start this notebook, you'll need to restart the notebook using `docker run` while giving it some additional arguments. You'll want to provide your FreeSurfer and/or HCP subject directory to the docker-image, and you'll want to provide a directory containing any data files to the docker as well.

Let's assume that you are using a FreeSurfer subject whose path is `/Volumes/server/Freesurfer_subjects/test-sub` and that the data for this subject is in a set of mgz files in the directory `/Users/nben/Data/retinotopy/test-sub`. You would want to start the docker-image with the following command.

```bash
> docker run --rm -it \
     -v /Volumes/server/Freesurfer_subjects/test-sub:/subjects/test-sub \
     -v /Users/nben/Data/retinotopy/test-sub:/data/test-sub \
     -v "${PWD}/work":/home/jovyan/work \
     -p 8765:8765 \
     nben/neuropythy:latest notebook
```

This makes the `/Volumes/server/Freesurfer_subjects/test-sub` directory available to this notebook as the `/subjects/test-sub` directory and the `/Users/nben/Data/retinotopy/test-sub` directory available as the `/data/test-sub` directory.

In [ ]:
# Edit these variables to be appropriate for the data you mounted

# The subject directory and the kind of subject it is
subject_path = '/subjects/test-sub'
subject_type = 'freesurfer' # or 'hcp'
#subject_type = 'hcp' # or 'freesurfer'

# The directory containing the pRF parameters
prf_path = '/data/test-sub'
# how the files are named:
prf_filename_patt = '{hemi}.prf_{prop}.mgz'
# the prf property names as keys, what are the {prop} taks in the file?
prf_props = {'polar_angle':        'angle',
             'eccentricity':       'eccen',
             'radius':             'prfsz',
             'variance_explained': 'vexpl'}
# We may also want to import inferred maps; we

## How do we measure cortical magnification?

In [6]:
import neuropythy as ny
import numpy as np